In [3]:
import pandas as pd 
from sqlalchemy import create_engine 
import numpy as np
import pymongo

### Extract the data

In [4]:
#Extracts the first sheet "2013-2020 Police Killings" from the xls spreadsheet

data_file = "data/OverallPoliceKillingData.xlsx"
police_data_df = pd.read_excel(data_file, sheet_name="2013-2020 Police Killings")
police_data_df.head()

Victim's name Victim's age Victim's gender     Victim's race  \
0             Chazz Hailey           22            Male  Pacific Islander   
1             Kevin Pulido           43            Male          Hispanic   
2           Matthew L. Fox           40            Male      Unknown race   
3  Name withheld by police           43            Male             Black   
4  Name withheld by police          NaN            Male      Unknown race   

  URL of image of victim Date of Incident (month/day/year)  \
0                    NaN                        2020-06-23   
1                    NaN                        2020-06-23   
2                    NaN                        2020-06-23   
3                    NaN                        2020-06-22   
4                    NaN                        2020-06-22   

            Street Address of Incident      City State  Zipcode  ...  \
0                1900 Pebblebrook Lane   Sherman    TX  75092.0  ...   
1       Mesa Avenue and Bragdon Avenue    Pueblo    CO  81004.0  ...   
2                     Blind Nick Drive   Wasilla    AK  99654.0  ...   
3        W Independence St & N Waco St  Giddings    TX  78942.0  ...   
4  East 35th Place and Sable Boulevard    Aurora    CO  80011.0  ...   

  Symptoms of mental illness? Unarmed/Did Not Have a Weapon  \
0                          No               Allegedly Armed   
1                          No               Allegedly Armed   
2                          No               Allegedly Armed   
3                          No               Allegedly Armed   
4                          No               Allegedly Armed   

  Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)  \
0                                              knife                                
1                                                gun                                
2                                                gun                                
3                                              knife                                
4                                                gun                                

  Alleged Threat Level (Source: WaPo) Fleeing (Source: WaPo)  \
0                              attack                    NaN   
1                              attack                    NaN   
2                                 NaN                    NaN   
3                              attack            Not fleeing   
4                              attack            Not fleeing   

  Body Camera (Source: WaPo) WaPo ID (If included in WaPo database)  \
0                        NaN                                    NaN   
1                         No                                 5943.0   
2                        NaN                                    NaN   
3                         No                                 5945.0   
4                        Yes                                 5938.0   

  Off-Duty Killing?  \
0               NaN   
1               NaN   
2               NaN   
3               NaN   
4               NaN   

  Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )  \
0                                                NaN                                                                                                                           
1                                                NaN                                                                                                                           
2                                                NaN                                                                                                                           
3                                                NaN                                                                                                                           
4                      

In [5]:
##Read csv file
dod_equipment_file = "data/dod_equipment.csv"
dod_equipment_df = pd.read_csv(dod_equipment_file)
dod_equipment_df.head()

State     Station Name (LEA)               NSN                    Item Name  \
0    AL  ABBEVILLE POLICE DEPT  6115-01-285-3012  GENERATOR SET,DIESEL ENGINE   
1    AL  ABBEVILLE POLICE DEPT  6230-01-439-3732      FLOODLIGHT SET,ELECTRIC   
2    AL  ABBEVILLE POLICE DEPT  7125-01-466-0952              CABINET,STORAGE   
3    AL  ABBEVILLE POLICE DEPT  2530-01-558-2138         WHEEL,PNEUMATIC TIRE   
4    AL  ABBEVILLE POLICE DEPT  2320-01-447-3892                TRUCK,WRECKER   

   Quantity        UI  Acquisition Value DEMIL Code  DEMIL IC Ship Date  \
0         2      Each            9922.25          A       7.0   00:00.0   
1         1      Each           12000.00          A       1.0   00:00.0   
2         4      Each             860.74          A       1.0   00:00.0   
3         1  Assembly            2015.00          A       1.0   00:00.0   
4         1      Each          331680.00          Q       6.0   00:00.0   

     state  
0  Alabama  
1  Alabama  
2  Alabama  
3  Alabama  
4  Alabama

In [6]:
##Read csv file
police_contracts_file = "data/police_contracts_dataset.csv"
police_contracts_df = pd.read_csv(police_contracts_file)
police_contracts_df.head()

City/State Expiration  Section  \
0  Albuquerque  7/16/2015   18.1.4   
1  Albuquerque  7/16/2015   18.1.5   
2  Albuquerque  7/16/2015  20.1.12   
3  Albuquerque  7/16/2015   20.1.3   
4  Albuquerque  7/16/2015   20.1.9   

                                     Policy Language  \
0  Derogatory material may be purged within twelv...   
1  "Human Resources Department files are a perman...   
2  When available, before an administrative inves...   
3  The name of the charging officer, complaintant...   
4  if the questioning is mechanically or stenogra...   

                               Category  \
0             Erases misconduct records   
1             Erases misconduct records   
2  Gives officers unfair access to info   
3  Gives officers unfair access to info   
4  Gives officers unfair access to info   

                           Specific Impact of Policy  Unique identifier  
0     Erases records at department head's discretion                 43  
1              Erases records if charges are dropped                 46  
2  Gives officers access to the names of complain...                180  
3  Gives officers access to the names of complain...                174  
4  Gives officers immediate access to the recorde...                516

### US Police Killings

In [7]:
##Rename columns so they can easily be queried
police_data_df.columns = ['victim_name','victim_age','victim_gender','victim_race','url_image_of_victim','date_of_incident',
                          'street_address_of_incident','city','state','zipcode','county','agency_responsible_for_death',
                          'cause_of_death','description_of_the_circumstances','official_disposition_of_death','criminal_charges',
                          'link_to_news_article_or_photo','symptoms_of_mental_illness','armed_or_unarmed','alleged_weapon',
                          'alleged_threat_level','fleeing','body_camera','wapo_id','off_duty_killing','suburban_rural_classification',
                         'id']
police_data_df.head()

victim_name victim_age victim_gender       victim_race  \
0             Chazz Hailey         22          Male  Pacific Islander   
1             Kevin Pulido         43          Male          Hispanic   
2           Matthew L. Fox         40          Male      Unknown race   
3  Name withheld by police         43          Male             Black   
4  Name withheld by police        NaN          Male      Unknown race   

  url_image_of_victim date_of_incident           street_address_of_incident  \
0                 NaN       2020-06-23                1900 Pebblebrook Lane   
1                 NaN       2020-06-23       Mesa Avenue and Bragdon Avenue   
2                 NaN       2020-06-23                     Blind Nick Drive   
3                 NaN       2020-06-22        W Independence St & N Waco St   
4                 NaN       2020-06-22  East 35th Place and Sable Boulevard   

       city state  zipcode  ... symptoms_of_mental_illness armed_or_unarmed  \
0   Sherman    TX  75092.0  ...                         No  Allegedly Armed   
1    Pueblo    CO  81004.0  ...                         No  Allegedly Armed   
2   Wasilla    AK  99654.0  ...                         No  Allegedly Armed   
3  Giddings    TX  78942.0  ...                         No  Allegedly Armed   
4    Aurora    CO  80011.0  ...                         No  Allegedly Armed   

  alleged_weapon alleged_threat_level      fleeing body_camera wapo_id  \
0          knife               attack          NaN         NaN     NaN   
1            gun               attack          NaN          No  5943.0   
2            gun                  NaN          NaN         NaN     NaN   
3          knife               attack  Not fleeing          No  5945.0   
4            gun               attack  Not fleeing         Yes  5938.0   

  off_duty_killing suburban_rural_classification  id  
0              NaN                           NaN NaN  
1              NaN                           NaN NaN  
2              NaN                           NaN NaN  
3              NaN                           NaN NaN  
4              NaN                           NaN NaN  

[5 rows x 27 columns]

In [8]:
##Update to include only columns needed for data analysis

police_cols = ["victim_name","victim_age","victim_gender","victim_race","url_image_of_victim","date_of_incident","street_address_of_incident",
               "city","state","zipcode","county","agency_responsible_for_death","cause_of_death","description_of_the_circumstances",
               "official_disposition_of_death","criminal_charges","link_to_news_article_or_photo","symptoms_of_mental_illness","armed_or_unarmed",
               "alleged_weapon","alleged_threat_level","fleeing","body_camera","off_duty_killing","suburban_rural_classification"]
police_df_cleaned = police_data_df[police_cols].copy()

In [9]:
police_df_cleaned.head()

victim_name victim_age victim_gender       victim_race  \
0             Chazz Hailey         22          Male  Pacific Islander   
1             Kevin Pulido         43          Male          Hispanic   
2           Matthew L. Fox         40          Male      Unknown race   
3  Name withheld by police         43          Male             Black   
4  Name withheld by police        NaN          Male      Unknown race   

  url_image_of_victim date_of_incident           street_address_of_incident  \
0                 NaN       2020-06-23                1900 Pebblebrook Lane   
1                 NaN       2020-06-23       Mesa Avenue and Bragdon Avenue   
2                 NaN       2020-06-23                     Blind Nick Drive   
3                 NaN       2020-06-22        W Independence St & N Waco St   
4                 NaN       2020-06-22  East 35th Place and Sable Boulevard   

       city state  zipcode  ...  criminal_charges  \
0   Sherman    TX  75092.0  ...  No known charges   
1    Pueblo    CO  81004.0  ...  No known charges   
2   Wasilla    AK  99654.0  ...  No known charges   
3  Giddings    TX  78942.0  ...  No known charges   
4    Aurora    CO  80011.0  ...  No known charges   

                       link_to_news_article_or_photo  \
0  https://www.heralddemocrat.com/news/20200623/t...   
1  https://www.chieftain.com/news/20200623/1-dead...   
2  https://www.ktva.com/story/42282005/man-dies-i...   
3  https://www.kxan.com/news/crime/several-people...   
4  https://www.denverpost.com/2020/06/22/aurora-p...   

  symptoms_of_mental_illness armed_or_unarmed alleged_weapon  \
0                         No  Allegedly Armed          knife   
1                         No  Allegedly Armed            gun   
2                         No  Allegedly Armed            gun   
3                         No  Allegedly Armed          knife   
4                         No  Allegedly Armed            gun   

  alleged_threat_level      fleeing body_camera off_duty_killing  \
0               attack          NaN         NaN              NaN   
1               attack          NaN          No              NaN   
2                  NaN          NaN         NaN              NaN   
3               attack  Not fleeing          No              NaN   
4               attack  Not fleeing         Yes              NaN   

  suburban_rural_classification  
0                           NaN  
1                           NaN  
2                           NaN  
3                           NaN  
4                           NaN  

[5 rows x 25 columns]

In [10]:
##Get data types
police_df_cleaned.dtypes

victim_name                                 object
victim_age                                  object
victim_gender                               object
victim_race                                 object
url_image_of_victim                         object
date_of_incident                    datetime64[ns]
street_address_of_incident                  object
city                                        object
state                                       object
zipcode                                    float64
county                                      object
agency_responsible_for_death                object
cause_of_death                              object
description_of_the_circumstances            object
official_disposition_of_death               object
criminal_charges                            object
link_to_news_article_or_photo               object
symptoms_of_mental_illness                  object
armed_or_unarmed                            object
alleged_weapon                 

In [11]:
##Get unique values for victim_age.  To understand what values are being captured
police_df_cleaned.victim_age.unique()

array([22, 43, 40, nan, 19, 35, 24, 25, 18, 38, 31, 50, 28, 27, 21, 23,
       63, 32, 42, 36, 59, 26, 37, 81, 39, 60, 44, 33, 65, 53, 49, 46, 34,
       52, 30, 58, 61, 56, 47, 57, 20, 48, 45, 69, 64, 29, 51, 54, 67, 16,
       17, 62, 66, 55, 41, 82, 73, 83, 75, 87, 76, 15, 68, 70, 74, 77, 80,
       14, 71, 'Unknown', 6, 78, 72, 84, 89, 1, '40s', 79, 91, 13, 86, 10,
       12, 95, 5, 93, 7, 107], dtype=object)

In [12]:
#Update Unknown and 40s to null value
police_df_cleaned['victim_age'].replace(['Unknown','40s'],[np.nan,np.nan],inplace=True) 
police_df_cleaned.victim_age.unique()

array([ 22.,  43.,  40.,  nan,  19.,  35.,  24.,  25.,  18.,  38.,  31.,
        50.,  28.,  27.,  21.,  23.,  63.,  32.,  42.,  36.,  59.,  26.,
        37.,  81.,  39.,  60.,  44.,  33.,  65.,  53.,  49.,  46.,  34.,
        52.,  30.,  58.,  61.,  56.,  47.,  57.,  20.,  48.,  45.,  69.,
        64.,  29.,  51.,  54.,  67.,  16.,  17.,  62.,  66.,  55.,  41.,
        82.,  73.,  83.,  75.,  87.,  76.,  15.,  68.,  70.,  74.,  77.,
        80.,  14.,  71.,   6.,  78.,  72.,  84.,  89.,   1.,  79.,  91.,
        13.,  86.,  10.,  12.,  95.,   5.,  93.,   7., 107.])

In [11]:
##Update index by 1 to be the unique value in the postgressql table
police_df_cleaned.index = range(1,len(police_df_cleaned)+1)

In [12]:
police_df_cleaned.head(10)

victim_name  victim_age victim_gender       victim_race  \
1                Chazz Hailey        22.0          Male  Pacific Islander   
2                Kevin Pulido        43.0          Male          Hispanic   
3              Matthew L. Fox        40.0          Male      Unknown race   
4     Name withheld by police        43.0          Male             Black   
5     Name withheld by police         NaN          Male      Unknown race   
6     Name withheld by police         NaN          Male      Unknown race   
7     Name withheld by police         NaN          Male      Unknown race   
8     Name withheld by police        19.0          Male      Unknown race   
9   Michael Kristopher Torres        35.0          Male          Hispanic   
10               Cody W. Cook        24.0          Male             White   

                                  url_image_of_victim date_of_incident  \
1                                                 NaN       2020-06-23   
2                                                 NaN       2020-06-23   
3                                                 NaN       2020-06-23   
4                                                 NaN       2020-06-22   
5                                                 NaN       2020-06-22   
6                                                 NaN       2020-06-22   
7                                                 NaN       2020-06-22   
8                                                 NaN       2020-06-21   
9   https://fatalencounters.org/wp-content/uploads...       2020-06-20   
10                                                NaN       2020-06-19   

             street_address_of_incident             city state  zipcode  ...  \
1                 1900 Pebblebrook Lane          Sherman    TX  75092.0  ...   
2        Mesa Avenue and Bragdon Avenue           Pueblo    CO  81004.0  ...   
3                      Blind Nick Drive          Wasilla    AK  99654.0  ...   
4         W Independence St & N Waco St         Giddings    TX  78942.0  ...   
5   East 35th Place and Sable Boulevard           Aurora    CO  80011.0  ...   
6                        628 W Craig Rd  North Las Vegas    NV  89032.0  ...   
7                       41945 Boardwalk      Palm Desert    CA  92211.0  ...   
8                130 North Temple Drive         Milpitas    CA  95035.0  ...   
9                  1067 Highland Street     Lake Charles    LA  70605.0  ...   
10                       6022 Boyd Road            Sodus    NY  14551.0  ...   

    criminal_charges                      link_to_news_article_or_photo  \
1   No known charges  https://www.heralddemocrat.com/news/20200623/t...   
2   No known charges  https://www.chieftain.com/news/20200623/1-dead...   
3   No known charges  https://www.ktva.com/story/42282005/man-dies-i...   
4   No known charges  https://www.kxan.com/news/crime/several-people...   
5   No known charges  https://www.denverpost.com/2020/06/22/aurora-p...   
6   No known charges  https://www.ktnv.com/news/man-threatening-suic...   
7   No known charges  https://kesq.com/news/breaking-news/2020/06/22...   
8   No known charges  https://www.mercurynews.com/2020/06/21/police-...   
9   No known charges  https://www.kplctv.com/2020/06/20/officer-invo...   
10  No known charges  https://www.rochesterfirst.com/news/sodus-man-...   

   symptoms_of_mental_illness armed_or_unarmed alleged_weapon  \
1                          No  Allegedly Armed          knife   
2                          No  Allegedly Armed            gun   
3                          No  Allegedly Armed            gun   
4                          No  Allegedly Armed          knife   
5                          No  Allegedly Armed            gun   
6                     Unknown  Allegedly Armed            gun   
7                          No          Unclear   undetermined   
8                          No  Allegedly Armed          knife   
9                          No  Allegedly Armed        machete  

### DOD  Equipment  Data

In [13]:
##Rename columns so they can easily be queried
dod_equipment_df.columns = ['state','station_name','nsn','item_name','quantity','ui','acquisition_value','demil_code','demil_ic','ship_date','state']
dod_equipment_df.head(10)

state           station_name               nsn  \
0    AL  ABBEVILLE POLICE DEPT  6115-01-285-3012   
1    AL  ABBEVILLE POLICE DEPT  6230-01-439-3732   
2    AL  ABBEVILLE POLICE DEPT  7125-01-466-0952   
3    AL  ABBEVILLE POLICE DEPT  2530-01-558-2138   
4    AL  ABBEVILLE POLICE DEPT  2320-01-447-3892   
5    AL  ABBEVILLE POLICE DEPT  5180-01-628-2375   
6    AL  ABBEVILLE POLICE DEPT  1240-01-411-1265   
7    AL  ABBEVILLE POLICE DEPT  4010-00-473-6166   
8    AL  ABBEVILLE POLICE DEPT  5855-DS-THR-MIMG   
9    AL  ABBEVILLE POLICE DEPT  1005-01-587-7175   

                       item_name  quantity        ui  acquisition_value  \
0    GENERATOR SET,DIESEL ENGINE         2      Each            9922.25   
1        FLOODLIGHT SET,ELECTRIC         1      Each           12000.00   
2                CABINET,STORAGE         4      Each             860.74   
3           WHEEL,PNEUMATIC TIRE         1  Assembly            2015.00   
4                  TRUCK,WRECKER         1      Each          331680.00   
5  TOOL KIT,AIRCRAFT MAINTENANCE         3       Kit            1298.00   
6                   SIGHT,REFLEX         9      Each             335.00   
7      CHAIN ASSEMBLY,SINGLE LEG        10      Each             160.18   
8    THERMAL IMAGINING EQUIPMENT         1      Each               0.00   
9                    MOUNT,RIFLE        10      Each            1647.00   

  demil_code  demil_ic ship_date    state  
0          A       7.0   00:00.0  Alabama  
1          A       1.0   00:00.0  Alabama  
2          A       1.0   00:00.0  Alabama  
3          A       1.0   00:00.0  Alabama  
4          Q       6.0   00:00.0  Alabama  
5          A       1.0   00:00.0  Alabama  
6          D       1.0   00:00.0  Alabama  
7          A       1.0   00:00.0  Alabama  
8          D       NaN   00:00.0  Alabama  
9          D       1.0   00:00.0  Alabama

In [14]:
dod_cols = ["state","station_name","nsn","item_name","quantity","ui","acquisition_value","demil_code","demil_ic","ship_date"]
dod_equipment_cleaned = dod_equipment_df[dod_cols].copy()

In [15]:
dod_equipment_cleaned.head()

state    state           station_name               nsn  \
0    AL  Alabama  ABBEVILLE POLICE DEPT  6115-01-285-3012   
1    AL  Alabama  ABBEVILLE POLICE DEPT  6230-01-439-3732   
2    AL  Alabama  ABBEVILLE POLICE DEPT  7125-01-466-0952   
3    AL  Alabama  ABBEVILLE POLICE DEPT  2530-01-558-2138   
4    AL  Alabama  ABBEVILLE POLICE DEPT  2320-01-447-3892   

                     item_name  quantity        ui  acquisition_value  \
0  GENERATOR SET,DIESEL ENGINE         2      Each            9922.25   
1      FLOODLIGHT SET,ELECTRIC         1      Each           12000.00   
2              CABINET,STORAGE         4      Each             860.74   
3         WHEEL,PNEUMATIC TIRE         1  Assembly            2015.00   
4                TRUCK,WRECKER         1      Each          331680.00   

  demil_code  demil_ic ship_date  
0          A       7.0   00:00.0  
1          A       1.0   00:00.0  
2          A       1.0   00:00.0  
3          A       1.0   00:00.0  
4          Q       6.0   00:00.0

In [16]:
dod_equipment_cleaned.dtypes

state                 object
state                 object
station_name          object
nsn                   object
item_name             object
quantity               int64
ui                    object
acquisition_value    float64
demil_code            object
demil_ic             float64
ship_date             object
dtype: object

In [17]:
##Update index by 1 to be the unique value in the postgressql table
dod_equipment_cleaned.index = range(1,len(dod_equipment_cleaned)+1)

In [18]:
dod_equipment_cleaned.head()

state    state           station_name               nsn  \
1    AL  Alabama  ABBEVILLE POLICE DEPT  6115-01-285-3012   
2    AL  Alabama  ABBEVILLE POLICE DEPT  6230-01-439-3732   
3    AL  Alabama  ABBEVILLE POLICE DEPT  7125-01-466-0952   
4    AL  Alabama  ABBEVILLE POLICE DEPT  2530-01-558-2138   
5    AL  Alabama  ABBEVILLE POLICE DEPT  2320-01-447-3892   

                     item_name  quantity        ui  acquisition_value  \
1  GENERATOR SET,DIESEL ENGINE         2      Each            9922.25   
2      FLOODLIGHT SET,ELECTRIC         1      Each           12000.00   
3              CABINET,STORAGE         4      Each             860.74   
4         WHEEL,PNEUMATIC TIRE         1  Assembly            2015.00   
5                TRUCK,WRECKER         1      Each          331680.00   

  demil_code  demil_ic ship_date  
1          A       7.0   00:00.0  
2          A       1.0   00:00.0  
3          A       1.0   00:00.0  
4          A       1.0   00:00.0  
5          Q       6.0   00:00.0

### Police Contracts data

In [19]:
##Rename columns so they can easily be queried
police_contracts_df.columns = ['city','expiration','section','policy_language','category','specific_impact_of_policy','id']
police_contracts_df.head(10)

city expiration   section  \
0  Albuquerque  7/16/2015    18.1.4   
1  Albuquerque  7/16/2015    18.1.5   
2  Albuquerque  7/16/2015   20.1.12   
3  Albuquerque  7/16/2015    20.1.3   
4  Albuquerque  7/16/2015    20.1.9   
5  Albuquerque  7/16/2015   20.1.10   
6  Albuquerque  7/16/2015    23.1.1   
7  Albuquerque  7/16/2015  20.1.5.1   
8  Albuquerque  7/16/2015    20.1.6   
9  Albuquerque  7/16/2015    20.1.6   

                                     policy_language  \
0  Derogatory material may be purged within twelv...   
1  "Human Resources Department files are a perman...   
2  When available, before an administrative inves...   
3  The name of the charging officer, complaintant...   
4  if the questioning is mechanically or stenogra...   
5  Any information released to the Police Oversig...   
6  Should a police officer be sued in a civil act...   
7  The interrogation shall be completed as soon a...   
8  Only two interrogators, the involved officer a...   
9  The officer shall not be subjected to any offe...   

                               category  \
0             Erases misconduct records   
1             Erases misconduct records   
2  Gives officers unfair access to info   
3  Gives officers unfair access to info   
4  Gives officers unfair access to info   
5           Limits oversight/discipline   
6      Requires city pay for misconduct   
7       Restricts/delays interrogations   
8       Restricts/delays interrogations   
9       Restricts/delays interrogations   

                           specific_impact_of_policy   id  
0     Erases records at department head's discretion   43  
1              Erases records if charges are dropped   46  
2  Gives officers access to the names of complain...  180  
3  Gives officers access to the names of complain...  174  
4  Gives officers immediate access to the recorde...  516  
5  Prevents the identity of the officer or inform...  404  
6  City pays costs of misconduct settlements and ...  337  
7  Limits interrogations to two 2-hour sessions w...  250  
8          Limits the number of interrogators to two  261  
9  Limits the type of language and incentives use...  275

In [20]:
#Extract needed columns
pc_cols = ["city","policy_language","category","specific_impact_of_policy"]
police_contracts_cleaned = police_contracts_df[pc_cols].copy()

In [21]:
police_contracts_cleaned.head()

city                                    policy_language  \
0  Albuquerque  Derogatory material may be purged within twelv...   
1  Albuquerque  "Human Resources Department files are a perman...   
2  Albuquerque  When available, before an administrative inves...   
3  Albuquerque  The name of the charging officer, complaintant...   
4  Albuquerque  if the questioning is mechanically or stenogra...   

                               category  \
0             Erases misconduct records   
1             Erases misconduct records   
2  Gives officers unfair access to info   
3  Gives officers unfair access to info   
4  Gives officers unfair access to info   

                           specific_impact_of_policy  
0     Erases records at department head's discretion  
1              Erases records if charges are dropped  
2  Gives officers access to the names of complain...  
3  Gives officers access to the names of complain...  
4  Gives officers immediate access to the recorde...

### Update data types

In [22]:
police_contracts_cleaned.dtypes

city                         object
policy_language              object
category                     object
specific_impact_of_policy    object
dtype: object

In [23]:
##Update index by 1 to be the unique value in the postgressql table
police_contracts_cleaned.index = range(1,len(police_contracts_cleaned)+1)

In [24]:
police_contracts_cleaned.head()

city                                    policy_language  \
1  Albuquerque  Derogatory material may be purged within twelv...   
2  Albuquerque  "Human Resources Department files are a perman...   
3  Albuquerque  When available, before an administrative inves...   
4  Albuquerque  The name of the charging officer, complaintant...   
5  Albuquerque  if the questioning is mechanically or stenogra...   

                               category  \
1             Erases misconduct records   
2             Erases misconduct records   
3  Gives officers unfair access to info   
4  Gives officers unfair access to info   
5  Gives officers unfair access to info   

                           specific_impact_of_policy  
1     Erases records at department head's discretion  
2              Erases records if charges are dropped  
3  Gives officers access to the names of complain...  
4  Gives officers access to the names of complain...  
5  Gives officers immediate access to the recorde...

## Create JSON files

In [25]:
police_df_cleaned.to_json(r'data/json/police.json')

police_contracts_cleaned.to_json(r'data/json/contracts.json')

In [26]:
dod_equipment_cleaned.columns = ['state', 'state_full', 'station_name', 'nsn', 'item_name', 'quantity', 'ui',
       'acquisition_value', 'demil_code', 'demil_ic', 'ship_date']


In [27]:
dod_equipment_cleaned.to_json(r'data/json/dod.json')

## Connect to MongoDB

In [28]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.police

## Create collections using dataframes

In [29]:
police = db["police"]
data = police_df_cleaned.to_dict('records')
police.insert_many(data)

In [30]:
contracts = db["contracts"]
data = police_contracts_cleaned.to_dict('records')
contracts.insert_many(data)

In [31]:
equipment = db["equipment"]
data = dod_equipment_cleaned.to_dict('records')
equipment.insert_many(data)


In [32]:
police.find_one({'state':'CA'})

{'_id': ObjectId('5efe92743cf5acbb47466188'),
 'victim_name': 'Name withheld by police',
 'victim_age': nan,
 'victim_gender': 'Male',
 'victim_race': 'Unknown race',
 'url_image_of_victim': nan,
 'date_of_incident': datetime.datetime(2020, 6, 22, 0, 0),
 'street_address_of_incident': '41945 Boardwalk',
 'city': 'Palm Desert',
 'state': 'CA',
 'zipcode': 92211.0,
 'county': 'Riverside',
 'agency_responsible_for_death': "Riverside County Sheriff's Department",
 'cause_of_death': 'Gunshot',
 'description_of_the_circumstances': "Reports of a robbery at C.V. Guns & Tackle came in at around 10:30 p.m. Deputies arrived after alarms were set off at the business. Upon arrival, deputies found a vehicle that had smashed into the business's front entrance. Shortly thereafter, an armed man came out of the business, and at least one deputy shot and killed him.",
 'official_disposition_of_death': 'Pending investigation/No Known Charges',
 'criminal_charges': 'No known charges',
 'link_to_news_articl